In [38]:
import pandas as pd
import numpy as np
import matplotlib as plt
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
movies_df = pd.read_csv('movies.csv',index_col='movieId')
links_df = pd.read_csv('links.csv', index_col='movieId')
ratings_df = pd.read_csv('ratings.csv', index_col='movieId')
tags_df = pd.read_csv('tags.csv', index_col='movieId')

In [3]:
genome_scoredf = pd.read_csv('genome-scores_large.csv')
genome_tagsdf = pd.read_csv('genome-tags_large.csv')
links_largedf = pd.read_csv('links_large.csv')
movies_largedf = pd.read_csv('movies_large.csv')
ratings_largedf = pd.read_csv('ratings_large.csv')
tags_largedf = pd.read_csv('tags_large.csv')

In [4]:
print ('small movies df shape:', movies_df.shape)
print ('small links df shape:' , links_df.shape)
print ('small ratings df shape:' , ratings_df.shape)
print ('small tags df shape:' , tags_df.shape)

small movies df shape: (9742, 2)
small links df shape: (9742, 2)
small ratings df shape: (100836, 3)
small tags df shape: (3683, 3)


In [5]:
print ('large genome scores df shape:', genome_scoredf.shape)
print ('large genome tags df shape:', genome_tagsdf.shape)
print ('large links df shape:', links_largedf.shape)
print ('large movies df shape:', movies_largedf.shape)
print ('large ratings df shape:', ratings_largedf.shape)
print ('large tags df shape:', tags_largedf.shape)

large genome scores df shape: (14862528, 3)
large genome tags df shape: (1128, 2)
large links df shape: (58098, 3)
large movies df shape: (58098, 3)
large ratings df shape: (27753444, 4)
large tags df shape: (1108997, 4)


In [6]:
movieratings_df = pd.merge(movies_df,ratings_df,left_index=True, right_index=True)

In [8]:
movieratings_df.head()

,title,genres,userId,rating,timestamp
movieId,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [13]:
movieratings_df.groupby(['movieId'])['rating'].mean()

movieId
1         3.920930
2         3.431818
3         3.259615
4         2.357143
5         3.071429
6         3.946078
7         3.185185
8         2.875000
9         3.125000
10        3.496212
11        3.671429
12        2.421053
13        3.125000
14        3.833333
15        3.000000
16        3.926829
17        3.776119
18        3.700000
19        2.727273
20        2.500000
21        3.494382
22        3.222222
23        3.125000
24        3.125000
25        3.625000
26        3.500000
27        3.333333
28        4.227273
29        4.013158
30        3.000000
31        3.184211
32        3.983051
34        3.652344
36        3.835821
38        2.125000
39        3.293269
40        4.250000
41        3.700000
42        3.000000
43        3.187500
44        2.543478
45        3.312500
46        3.066667
47        3.975369
48        3.147059
49        3.000000
50        4.237745
52        3.464286
53        5.000000
54        2.333333
55        4.000000
57        3.062500
58  

In [35]:
mr_valuecounts = movieratings_df.groupby(['movieId'])['rating'].count()

In [39]:
mr_valuecounts.sort_values(ascending=False)

movieId
356       329
318       317
296       307
593       279
2571      278
260       251
480       238
110       237
589       224
527       220
2959      218
1         215
1196      211
50        204
2858      204
47        203
780       202
150       201
1198      200
4993      198
1210      196
858       192
457       190
592       189
2028      188
5952      188
7153      185
588       183
608       181
2762      179
380       178
32        177
364       172
1270      171
377       171
3578      170
4306      170
1580      165
590       164
648       162
344       161
4226      159
367       157
58559     149
6539      149
595       146
1214      146
1036      145
165       144
500       144
1265      143
79132     143
1197      142
6377      141
1704      141
2628      140
1721      140
316       140
1291      140
153       137
1136      136
597       135
3793      133
293       133
1193      133
231       133
4886      132
10        132
1240      131
6874      131
1089      13

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('dark')
%matplotlib inline

plt.figure(figsize=(8,6))
plt.rcParams['patch.force_edgecolor'] = True
ratings_mean_count['rating_counts'].hist(bins=50)